# IBM Capstone Proyect: Week 3, Part I

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# Using Beautifulsoup to collect data

In [3]:
#Create a beautifulsoup object (soup) from the source.
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

# Find the Table in the soup...
table = soup.find('table', class_ = 'wikitable') 
#print(table)

When I print the Table, all the rows are separeted with '< tr >' , so we can exctract that information.

In [4]:
Rows = table.find_all('tr')

Pc = [] # Postal Code
Borough = []
Neigh = []
for k in Rows[1::]:
    row = k.find_all('td')
    #I use .text.split('\n')[0] to keep the text before the \n
    Pc.append(row[0].text.split('\n')[0])
    Borough.append(row[1].text.split('\n')[0])
    Neigh.append(row[2].text.split('\n')[0])


# Clean Data Frame

It is easier to drop the Not assigned cells with pandas after collect all the data

In [5]:
New_Neigh = []

# Replace("/", ",") and create a New list New_Neigh to store it
for N in Neigh:
    new_string = N.replace("/", ",")
    New_Neigh.append(new_string)

# Create a Dataframe and fill it with the information    
df = pd.DataFrame()
df['PostalCode'] = Pc
df['Borough'] = Borough
df['Neighborhood'] = New_Neigh 

# Drop de rows where df['Borough'] =='Not assigned'
indexBorough = df[ df['Borough'] =='Not assigned' ].index
df.drop(indexBorough , inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [6]:
# Fill where df['Neighborhood'] ==''
df.loc[df["Neighborhood"] == '','Neighborhood'] = df["Borough"]

In [7]:
#Save the file and return the shape of the Dataframe
df.reset_index(drop=True, inplace=True)
df.to_csv('Toronto_PC.csv')
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
df.shape

(103, 3)

# Part 2

I download the csv provided by the course of the geospatial coordinates: https://cocl.us/Geospatial_data

In [12]:
df_longlat = pd.read_csv('Geospatial_Coordinates.csv')
df_longlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I have to rename the Postal Code Column of df_longlat to match with the PostalCode column of df

In [14]:
df_longlat.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_longlat.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now, I merge the two tables on the column "PostalCode"

In [15]:
df_complete = df.merge(df_longlat, on="PostalCode", how="left")
df_complete.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
